In [9]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_from_disk
from llama_index.core import Document 
from llama_index.core.node_parser import LangchainNodeParser
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pymilvus import MilvusClient

## PreProcess

In [2]:
df = pd.read_csv('podcastdata_dataset.csv')
def preprocess_tolower(text):
    text=text.lower()
    return text

df['text']=df['text'].apply(preprocess_tolower)

## Splitter

In [4]:
docs = [
    Document(
        text=row['text'],
        metadata={
            'Title': row['title'],
            'Guest': row['guest'],
            'id': row['id']
        }
    )
    for _, row in df[:3].iterrows()
]

parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap = 200
    )
)
nodes = parser.get_nodes_from_documents(docs)


## Embedding model

In [5]:
# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
test_emeds = embed_model.get_text_embedding("Hello World!")
embedding_dim = len(test_emeds)

d:\Projects\HYDE\.conda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def gen_embedding(node: str):
    embedd = embed_model.get_text_embedding(node.text)
    node.embedding = embedd

## Milvus DB

In [12]:
milvus_client = MilvusClient("./milvus_demo.db")
collection_name = "lex_transcript_collection"

Failed to create new connection using: 1f23df85fbae421eb8888bef34708bb8


ConnectionConfigException: <ConnectionConfigException: (code=1, message=Illegal uri: [./milvus_demo.db], expected form 'http[s]://[user:password@]example.com[:12345]')>